In [9]:
import csv
import os
import torch
import torchaudio
import librosa
from cnn_model import CNNModel

from datapreprocessing import AudioProcessor
from torch.utils.data import Dataset



In [19]:
# audio_file_path = "/Users/zainhazzouri/Desktop/egp1.mp3"
audio_file_path = "/Users/zainhazzouri/projects/Bachelor_Thesis/Data/test/music_wav/beatles.wav"

SAMPLE_RATE = 22050 # sample rate of the audio file
bit_depth = 16 # bit depth of the audio file
hop_length = 512
n_mfcc = 20 # number of MFCCs features
n_fft=1024, # window size
n_mels = 64 # number of mel bands to generate
win_length = None # window length



In [20]:
# Set device
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_built():  # if you have apple silicon mac
    device = "mps"  # if it doesn't work try device = torch.device('mps')
else:
    device = "cpu"
print(f"Using {device}")


Using mps


In [21]:
model = CNNModel().to(device)
model.load_state_dict(torch.load("best_model.pth"))
model.eval() # some parts are turned off for evaluation because we don't need them

CNNModel(
  (Maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv1): conv_block(
    (conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (Conv2): conv_block(
    (conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
  

In [22]:
def split_waveform(waveform, sample_rate):
    segment_length = sample_rate # 1 second segments
    num_segments = waveform.shape[-1] // segment_length
    segments = []

    for i in range(num_segments):
        start = i * segment_length
        end = start + segment_length
        segments.append(waveform[:, start:end])

    return segments

In [23]:
audio_processor = AudioProcessor(audio_file_path)


In [24]:
def classify_audio_file_segments(audio_file_path, audio_processor):
    file_ext = os.path.splitext(audio_file_path)[1].lower() # file extension

    if file_ext == '.mp3':
        waveform, sample_rate = librosa.load(audio_file_path, sr=SAMPLE_RATE)
        waveform = torch.from_numpy(waveform).unsqueeze(0)
    else:
        waveform, sample_rate = torchaudio.load(audio_file_path)

    segments = split_waveform(waveform, sample_rate)

    segment_classifications = []

    for segment in segments:
        # Apply the sequence of operations
        segment = audio_processor._resample_if_necessary(segment, sample_rate)
        segment = audio_processor._mix_down_if_necessary(segment)
        segment = audio_processor._right_pad_if_necessary(segment)
        segment = audio_processor._cut_if_necessary(segment)

        # Apply the MFCC transformation directly
        mfcc = torchaudio.transforms.MFCC(sample_rate=sample_rate, n_mfcc=audio_processor.n_mfcc)(segment)
        mfcc = mfcc.to(audio_processor.device).unsqueeze(0)
        output = model(mfcc)
        _, predicted_class = torch.max(output, 1)
        segment_classifications.append(predicted_class.item())

    return segment_classifications

In [26]:

def format_time(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


def generate_classification_table(classification_results, segment_duration=1):
    table = []
    current_label = classification_results[0]
    start_time = 0

    for i, label in enumerate(classification_results[1:], 1):
        if label != current_label:
            table.append([format_time(start_time), format_time(i * segment_duration), current_label])
            start_time = i * segment_duration
            current_label = label

    table.append([format_time(start_time), format_time(len(classification_results) * segment_duration), current_label])

    return table


def save_classification_table_to_csv(table, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Start Time', 'End Time', 'Class'])
        for row in table:
            csvwriter.writerow(row)

# print(audio_file_path)
classification_results = classify_audio_file_segments(audio_file_path, audio_processor)
table = generate_classification_table(classification_results)

# Save the table to a CSV file
save_classification_table_to_csv(table, "classification_table.csv")

# Print the table
print("Start Time | End Time | Class")
print("-" * 28)
for row in table:
    print(f"{row[0]} | {row[1]} | {row[2]}")


# music_wav = 0, speech_wav = 1, Mix_wav = 2, silence_wav = 3


Start Time | End Time | Class
----------------------------
00:00:00 | 00:00:07 | 1
00:00:07 | 00:00:44 | 2
00:00:44 | 00:21:16 | 1
00:21:16 | 00:21:17 | 2
00:21:17 | 00:21:19 | 1
00:21:19 | 00:21:20 | 2
00:21:20 | 00:26:50 | 1
00:26:50 | 00:26:51 | 2
00:26:51 | 00:42:05 | 1
00:42:05 | 00:42:06 | 2
00:42:06 | 00:43:25 | 1
00:43:25 | 00:43:26 | 2
00:43:26 | 00:43:45 | 1
00:43:45 | 00:43:46 | 2
00:43:46 | 00:44:13 | 1
00:44:13 | 00:44:14 | 2
00:44:14 | 00:44:20 | 1
00:44:20 | 00:44:21 | 2
00:44:21 | 00:44:49 | 1
00:44:49 | 00:44:50 | 0
00:44:50 | 00:45:31 | 1
00:45:31 | 00:45:32 | 2
00:45:32 | 00:46:39 | 1
00:46:39 | 00:46:40 | 2
00:46:40 | 00:47:00 | 1
00:47:00 | 00:47:01 | 2
00:47:01 | 00:47:59 | 1
00:47:59 | 00:48:00 | 2
00:48:00 | 00:49:41 | 1
00:49:41 | 00:49:42 | 2
00:49:42 | 00:55:30 | 1
00:55:30 | 00:55:31 | 2
00:55:31 | 00:59:21 | 1
00:59:21 | 00:59:23 | 2
00:59:23 | 01:01:10 | 1
01:01:10 | 01:01:11 | 2
01:01:11 | 01:01:33 | 1
01:01:33 | 01:01:34 | 2
01:01:34 | 01:04:30 | 1


In [27]:
audio_processor = AudioProcessor(audio_file_path)

classification_results = classify_audio_file_segments(audio_file_path, audio_processor)
table = generate_classification_table(classification_results)
print(table)



[['00:00:00', '00:00:07', 1], ['00:00:07', '00:00:44', 2], ['00:00:44', '00:21:16', 1], ['00:21:16', '00:21:17', 2], ['00:21:17', '00:21:19', 1], ['00:21:19', '00:21:20', 2], ['00:21:20', '00:26:50', 1], ['00:26:50', '00:26:51', 2], ['00:26:51', '00:42:05', 1], ['00:42:05', '00:42:06', 2], ['00:42:06', '00:43:25', 1], ['00:43:25', '00:43:26', 2], ['00:43:26', '00:43:45', 1], ['00:43:45', '00:43:46', 2], ['00:43:46', '00:44:13', 1], ['00:44:13', '00:44:14', 2], ['00:44:14', '00:44:20', 1], ['00:44:20', '00:44:21', 2], ['00:44:21', '00:44:49', 1], ['00:44:49', '00:44:50', 0], ['00:44:50', '00:45:31', 1], ['00:45:31', '00:45:32', 2], ['00:45:32', '00:46:39', 1], ['00:46:39', '00:46:40', 2], ['00:46:40', '00:47:00', 1], ['00:47:00', '00:47:01', 2], ['00:47:01', '00:47:59', 1], ['00:47:59', '00:48:00', 2], ['00:48:00', '00:49:41', 1], ['00:49:41', '00:49:42', 2], ['00:49:42', '00:55:30', 1], ['00:55:30', '00:55:31', 2], ['00:55:31', '00:59:21', 1], ['00:59:21', '00:59:23', 2], ['00:59:23', 